## XML-Struktur
fra Ellens mail

All kildetekst ligger innenfor `<div type="chapter" subtype="source">`.
Alle innledninger og annen utgivertekst ligger i `<div type="chapter" subtype="intro">`, disse kan nok undertrykkes. Det samme gjelder illustrasjoner som er kodet i `<div type="chapter" subtype="illustration">`
 
Innenfor div ligger kildetekst hovedsakelig i `<head>, <p> og/eller <lg>/<l>`.  Og innenfor alle disse kan det forekomme spesialkoder som `<dateline>, <signed>, <del>, <add>, <unclear>, <gap/>, <note>, <pb/>, <hi> og <emph>`.  Noen ganger forekommer underkapitler, under-<div>’er, som også kan inneholde alt det nevnte. Så hvis du har en kommando som sier ta med all tekst innenfor en `<div type="chapter" subtype="source">` uavhengig av «child»/undernivå hadde nok det funket fint…..
 
Når det gjelder metadata, så kan du nok bruke `<titleStmt>` som ligger i `<fileDesc> i <teiHeader>` til noe, kodingen i `<titleStmt>` ser slik ut:
```<titleStmt>
<title>Brevveksling 1874&#x2013;1908</title>
<author>Kitty L. Kielland og Eilif Peterssen</author>
<editor>Utgitt ved Anne Melg&#x00E5;rd, NB kilder 8:2, Nasjonalbiblioteket, Oslo 2019</editor>
</titleStmt>```

In [87]:
import dhlab.nbtext as nb
from dhlab.nbtokenizer import tokenize
from collections import Counter
import pandas as pd

In [18]:
import zipfile
from bs4 import BeautifulSoup
import sys
import zipfile
import re

In [20]:
def get_files_from_zip(document):
    """Find all documents in .zip"""
    import sys
    import zipfile
    import re
    from bs4 import BeautifulSoup

    with zipfile.ZipFile(document, 'r') as zfp:
           n = zfp.namelist()
    return n

In [70]:
def file_contents(name, zipfolder):
    
    with zipfile.ZipFile(zipfolder, 'r') as zfp:
        try:
            with zfp.open(name) as fp:
                soup = fp.read()
        except:
            print('problemer', name)
    return soup

In [72]:
def extract_text(file, zipfolder):
    text = []
    soup = BeautifulSoup(file_contents(file, zipfolder), 'xml')
    for node in soup.find_all('div', {"type":"chapter", "subtype":"source"}):
        for p in node.find_all(['p', 'head']) or p in node_fi:
            text.append(tokenize(p.text))
    return text

In [75]:
zipfiles = get_files_from_zip("XML_NBkilder_20191112.zip")

Se på en og en fil i første omgang, for å ut teksten

In [105]:
def make_corpus(zipfolder):
    texts = dict()
    for f in zipfiles:
        texts[f] = extract_text(f,zipfolder)
    return texts

In [104]:
def make_dtm(texts):
    dtm = pd.DataFrame()
    freqs = dict()
    for text in texts.keys():
        print(text)
        c = Counter()
        for p in texts[text]:
            c.update(p)
        freqs[text] = nb.frame(c, text)
    dtm = pd.concat([freqs[text] for text in freqs.keys()], axis=1, sort=False)
    return dtm

In [207]:
def sublist(phrase, alist):
    if phrase == [] or alist == []:
        return None
    found = None
    try:
        ix = alist.index(phrase[0])
        searching = True
        while ix < len(alist) and searching == True:
            if alist[ix: ix + len(phrase)] == phrase:
                if found == None:
                    found = [ix]
                else:
                    found.append(ix)
                try:
                    sublist = alist[ix + len(phrase):]
                    ix += sublist.index(phrase[0])
                except:
                    searching = False
            else:
                ix += 1
    except:
        True
    return found

In [201]:
sublist([2], [2,3,4,5, 2,3,4,6,6,6,6,2,3,4])

[0, 4, 11]

In [202]:
[2,3,4,5, 2,3,4,6,6,6,6,2,3,4][12:14]

[3, 4]

In [180]:
def konk(phrase, corpus):
    
    return [[p,sublist(phrase, p)] for t in corpus for p in t ]

In [205]:
def konk(phrase, corpus):
    finds = dict()
    for t in corpus:
        finds[t] = []
        for i in range(len(corpus[t])):
            res = sublist(phrase, corpus[t][i])
            if res != None:
                finds[t].append((i, res))
    return finds

def konk_or(phrases, corpus):
    konks = dict()
    for phrase in phrases:
        konks[phrase] = konk(phrase, corpus)
    for i in corpus

In [211]:
konk('sel'.split(), texts)

{'NBkilder_Amundsen_Slededagbok.xml': [(19, [82]), (29, [93]), (31, [29])],
 'NBkilder_Collett_Brev_1841-51.xml': [],
 'NBkilder_Collett_Brev_til_Bjornson.xml': [],
 'NBkilder_Collett_Brev_til_Ibsen.xml': [],
 'NBkilder_Collett_Brev_til_Lie.xml': [],
 'NBkilder_Collett_Skildpadde.xml': [],
 'NBkilder_Dardel_Voyage.xml': [],
 'NBkilder_Grunnlovskilder1814.xml': [],
 'NBkilder_Kjerulf_1833.xml': [],
 'NBkilder_Kjerulf_1840.xml': [],
 'NBkilder_Kjerulf_1850.xml': [],
 'NBkilder_Kjerulf_1851.xml': [],
 'NBkilder_Kjerulf_1852.xml': [],
 'NBkilder_KKielland_AGarborg_brev.xml': [],
 'NBkilder_KKielland_DSkavlan_brev.xml': [],
 'NBkilder_KKielland_EPeterssen_brev.xml': [],
 'NBkilder_Koren_1798.xml': [],
 'NBkilder_Koren_1805.xml': [],
 'NBkilder_Koren_1808.xml': [],
 'NBkilder_Koren_1809.xml': [],
 'NBkilder_Koren_1810.xml': [],
 'NBkilder_Koren_1811.xml': [],
 'NBkilder_Koren_1812.xml': [],
 'NBkilder_Koren_1813.xml': [],
 'NBkilder_Koren_1814.xml': [],
 'NBkilder_Koren_1815.xml': [],
 'NBki

In [101]:
dtm = dtm.fillna(0)

In [103]:
nb.frame_sort(dtm, 'NBkilder_Koren_1815.xml').head(50).style.background_gradient()